# Young Tableau
This is a shape class that allows you to make and find all the standard young tableau 
combinations of a given shape. Feel free to read the docstrings to get a better idea of what you can do. 
I'm not exactly of the applications of this yet but I thought the tableau permutation thing was a pretty cool concept.
I may add a diagram that shows the proof of the hook length formula if I have time/interest. (google drawings)

This is what a standard young tableau would look like.

<img src="files/data/Young_tableaux_for_541_partition.svg">
<a href="https://commons.wikimedia.org/wiki/File:Young_tableaux_for_541_partition.svg#/media/File:Young_tableaux_for_541_partition.svg">Young tableaux for 541 partition</a>". Licensed under <a href="http://creativecommons.org/licenses/by-sa/3.0/" title="Creative Commons Attribution-Share Alike 3.0">CC BY-SA 3.0</a> via <a href="https://commons.wikimedia.org/wiki/">Commons</a>

In [1]:
import numpy as np
from functools import reduce
from operator import mul
from math import factorial
from copy import deepcopy
import os

class Shape(object):
    """
        A shape object which is really just a 2d array holding integers.
        Was just messing around with static and normal python methods.
    """
    
    def __init__(self, matrix):
        self.__shape = np.array(matrix)
    
    @property
    def shape(self):
        return self.__shape
    
    def __str__(self):
        return str(self.__shape)
    
    def display(self):
        """
            Print out the shape.
        """
        for row in self.__shape:
            print(row)
        
    @staticmethod
    def isOccupiedCell(cell):
        """
            Checks if a cell is 'occupied' which means it contains an integer
            in the case someone uses x's or something to fill in non-filled areas.
            I don't know... I just made it this way.
        """
        try:
            int(cell)
            return True
        except ValueError:
            return False
    
    @staticmethod
    def isArrayIncreasing(arr):
        """
            Checks that a given array is increasing (strict).
        """
        return all(x < y for x, y in zip(arr, arr[1:]))
    
    @staticmethod
    def getColumn(shape, index):
        """
            Gets all the values in a column of an array by an index. [0,len-1]
        """
        column = []
        for row in shape:
            for ix, cell in enumerate(row):
                if ix == index:
                    column.append(cell)
        return column             
    
    def _getValueSet(self):
        """
            Gets a set of all values in the shape.
        """
        return set(np.concatenate(self.__shape))  # flatten doesn't work on arrays of variable length
    
    def isLeftJustified(self):
        """
            Checks that the shape is left justfied. Basically just have
            to check if in any row there is a sequence of 
            occupied -> empty -> occupied. (then it is not left justified).
        """
        for row in self.__shape:
            can_be_empty = True
            for cell in row:
                occupied = self.isOccupiedCell(cell)
                if not occupied:
                    can_be_empty = False
                if not can_be_empty and occupied:
                    return False
        return True
    
    def __getRowLengths(self):
        """
            Gets the length of all rows in the shape.
        """
        return [len(list(filter(self.isOccupiedCell, row))) for row in self.__shape]
    
    def rowLengthDecreasingOrEqual(self):
        """
            Checks that all row lengths from top to bottom are decreasing
            or equal to the previous one.
        """
        arr = self.__getRowLengths()
        return all(x >= y for x, y in zip(arr, arr[1:]))
            
    def rowsIncreasing(self):
        """
            Checks that all rows contain increasing numbers.
        """
        for row in self.__shape:
            if not self.isArrayIncreasing(list(filter(self.isOccupiedCell, row))):
                return False
        return True
    
    def columnsIncreasing(self):
        """
            Checks if the values in each column are increasing.
            (Written after getColumn. woops.)"""
        for column in self.__shape.T:
            if not self.isArrayIncreasing(list(filter(self.isOccupiedCell, column))):
                return False
        return True    
    
    def validNumbers(self):
        """
            Checks if the numbers in the shape are consecutive integers
            from [1,n].
        """
        numbers = []
        max_num = 0
        for row in self.__shape:
            for cell in row:
                if cell > max_num:
                    max_num = cell
                numbers.append(cell)
        return sorted(numbers) == list(range(1,int(max_num+1)))
    
    def isSYT(self):
        """
            Checks the shape is a Standard Young Tableau.
            Defined as a shape which is 
                - left justified 
                - whose row length is decreasing (non-strict)
                - whose numbers in each row are increasing left to right (strict)
                - whose numbers in each column are increasing from top to bottom (strict)
                - which includes all integers [1,n]
        """
        return (self.isLeftJustified() 
                and self.rowLengthDecreasingOrEqual() 
                and self.rowsIncreasing() 
                and self.columnsIncreasing()
                and self.validNumbers())
    
    def numberOfOccupiedCells(self):
        """
            Returns the number of occupied cells
            in a shape.
        """
        return sum(self.__getRowLengths())
    
    def getHookNumber(self, cell_row_ix, cell_column_ix):
        """
            Calculates the hook number of a shape which is the number
            of the cells to the right of the cell, plus the cell itself,
            plus the number of cells below it.
        """
        shape = self.__shape
        return (len(list(filter(self.isOccupiedCell, shape[cell_row_ix]))) - (cell_column_ix+1) +
        (len(list(filter(self.isOccupiedCell, shape.T[cell_column_ix]))) - cell_row_ix))
    
    def numberOfSYTPermutations(self):
        """
            The number of SYT permutations can be calculated by the factorial
            of the number of cells in the shape divided by the product of the 
            hook numbers of the shape.
        """
        if not self.isSYT():
            return None
        
        hook_numbers = []
        for row_ix, row in enumerate(self.__shape):
            for column_ix, cell in enumerate(row):
                if self.isOccupiedCell(cell):
                    hook_numbers.append(int(self.getHookNumber(row_ix, column_ix)))
        return factorial(self.numberOfOccupiedCells()) / reduce(mul, hook_numbers, 1)
    
    def getShapeHookNumbers(self):
        """
            This returns a shape where the hook numbers of each cell replace
            the numbers that were originally in it.
        """
        hook_numbers = []
        for row_ix, row in enumerate(self.__shape):
            temp = []
            for column_ix, cell in enumerate(row):
                if self.isOccupiedCell(cell):
                    temp.append(int(self.getHookNumber(row_ix, column_ix)))
            hook_numbers.append(temp)
        return hook_numbers
    
    def generateAllSYTPermutations(self):
        """
            This functions seems a bit hackish which it sort-of is.
            I didn't really take the time to figure out how to either yield
            or wrap the makeShapes function in another function because it 
            is a weird blend of iteration and recursion. I first build the shape
            as a 1 in the top left corner (given) and then fill the rest of the shape
            with zeros. Then I find the valid locations that the next number can go to
            (kept track of by counter) and then iterate over the slightly-more-filled-in
            shapes and recursively fill those. It checks that all the calculated outcomes
            are SYT's and that the number of results is equal the to the expected number
            of SYT's.
        """
        
        if not self.isSYT():
            return None
        
        given = []
        for length in self.__getRowLengths():
            given.append(np.zeros(length))
        
        given[0][0] = 1
        
        results = []
        def makeShapes(shapes, counter):  # my beautiful function...
            if counter > self.numberOfOccupiedCells()-1:        
                results.append(shapes)  # shape filled at this point
            else:
                for shape in shapes:
                    makeShapes(Shape._fillLocations(shape,Shape._findValidNextLocations(shape),counter+1), counter+1)
        
        makeShapes([given], 1)  # this is bad, I know...
        results = [x[0] for x in results]
        
        assert(len(results) == self.numberOfSYTPermutations())
        assert(all(Shape(s).isSYT() for s in results))
        return results
    
    def displayAllSYTPermutations(self):
        """
            This function prints all of the possible permutations
            of the SYT.
        """
        if not self.isSYT():
            return None
        for syt in self.generateAllSYTPermutations():
            Shape(syt).display()
            print()
            
    def saveSYTPermutationsToFile(self, filepath, header=True):
        """
            This function saves all the permutations of the SYT
            to a file. The header can be removed. It just puts
            the number of possible permutations on the top of the 
            file.
        """
        if not self.isSYT():
            return None
        
        path = os.path.expanduser(filepath)
        with open(path, 'w') as f:
            if header:
                f.write("Number of SYT permutations: " + str(self.numberOfSYTPermutations()) + "\n")
            for syt in self.generateAllSYTPermutations():
                for row in syt:
                    f.write(np.array_str(row) + "\n")
                f.write("\n")
    
    @staticmethod
    def _fillLocations(shape, locations, number):
        """
            Given a 2d array, coordinates in the array and a number
            this functions returns all of the possible shapes formed
            by putting the number in each location.
        """
        shapes = []
        for x,y in locations:
            temp = deepcopy(shape)
            temp[x][y] = number
            shapes.append(temp)
        return shapes
    
    @staticmethod
    def _findValidNextLocations(shape):
        """
            Given a 2d array this function will find the valid
            locations to put the next value. Since you build all
            of the SYT's sequentially you just need to find the 
            next position in the sequence.
        """
        # only adjacent down and to the right
        open_adjacents = set()
        # (row,col) for -> shape[row][col]
        for x, row in enumerate(shape):
            for y, cell in enumerate(row):
                if cell != 0:            
                    if x+1 < len(Shape.getColumn(shape, y)) and shape[x+1][y] == 0:
                        open_adjacents.add((x+1,y))
                    if y+1 < len(row) and shape[x][y+1] == 0:
                        open_adjacents.add((x,y+1))    
        if open_adjacents == set():
            return None
        removals = set()
        for adj in open_adjacents:
            for adj2 in open_adjacents:
                if adj[0] == adj2[0] and adj[1] < adj2[1]:
                    removals.add(adj2)
                if adj[1] == adj2[1] and adj[0] < adj2[0]:
                    removals.add(adj2)
        
        return open_adjacents.difference(removals)

In [2]:
# Create our shape (this one is a SYT)
shape = Shape([[1,2,3,4],[5,6,7],[8,9],[10]])
print("Standard Young Tableau")
shape.display()
print("Is a STY?: " + str(shape.isSYT()))

Standard Young Tableau
[1, 2, 3, 4]
[5, 6, 7]
[8, 9]
[10]
Is a STY?: True


In [3]:
print("Number of SYT permutations: " + str(shape.numberOfSYTPermutations()))

Number of SYT permutations: 768.0


In [4]:
print("Shape hook numbers:")
for x in shape.getShapeHookNumbers():
    print(x)

Shape hook numbers:
[7, 5, 3, 1]
[5, 3, 1]
[3, 1]
[1]


In [5]:
# Find all permutations of the SYT and save it to a file
shape.saveSYTPermutationsToFile("~/output.txt")

In [6]:
print("All permutations of standard tableau for shape:") 
shape.displayAllSYTPermutations()
print("===============================================")

All permutations of standard tableau for shape:
[ 1.  2.  7.  9.]
[ 3.  6.  8.]
[  4.  10.]
[ 5.]

[  1.   2.   7.  10.]
[ 3.  6.  8.]
[ 4.  9.]
[ 5.]

[ 1.  2.  7.  8.]
[ 3.  6.  9.]
[  4.  10.]
[ 5.]

[ 1.  2.  7.  8.]
[  3.   6.  10.]
[ 4.  9.]
[ 5.]

[  1.   2.   7.  10.]
[ 3.  6.  9.]
[ 4.  8.]
[ 5.]

[ 1.  2.  7.  9.]
[  3.   6.  10.]
[ 4.  8.]
[ 5.]

[  1.   2.   8.  10.]
[ 3.  6.  9.]
[ 4.  7.]
[ 5.]

[ 1.  2.  8.  9.]
[  3.   6.  10.]
[ 4.  7.]
[ 5.]

[ 1.  2.  6.  7.]
[ 3.  8.  9.]
[  4.  10.]
[ 5.]

[ 1.  2.  6.  7.]
[  3.   8.  10.]
[ 4.  9.]
[ 5.]

[ 1.  2.  6.  9.]
[ 3.  7.  8.]
[  4.  10.]
[ 5.]

[  1.   2.   6.  10.]
[ 3.  7.  8.]
[ 4.  9.]
[ 5.]

[ 1.  2.  6.  8.]
[ 3.  7.  9.]
[  4.  10.]
[ 5.]

[ 1.  2.  6.  8.]
[  3.   7.  10.]
[ 4.  9.]
[ 5.]

[  1.   2.   6.  10.]
[ 3.  7.  9.]
[ 4.  8.]
[ 5.]

[ 1.  2.  6.  9.]
[  3.   7.  10.]
[ 4.  8.]
[ 5.]

[ 1.  2.  7.  9.]
[ 3.  5.  8.]
[  4.  10.]
[ 6.]

[  1.   2.   7.  10.]
[ 3.  5.  8.]
[ 4.  9.]
[ 6.]

[ 1.  2.  7.  8.

In [21]:
s = set()
for x in shape.generateAllSYTPermutations():
    s.add(x[3][0])
print(s)

{4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0}
